In [104]:
#standard imports. Note: You must pip install nasdaqdatalink 1st

import os
import pandas as pd
import nasdaqdatalink
import requests
from pathlib import Path
import shutil

In [105]:
# Linking my API key to .env in the same folder. The key is stored in the folder without any quotations around it 

nasdaqdatalink.read_key(filename=".env")

In [106]:
#A function to retrieve a dataframe of counties, zips, etc
def get_regions(regions):
    region_df=nasdaqdatalink.get_table('ZILLOW/REGIONS', region_type=regions)  
    return region_df

In [107]:
# Using get_regions to retrieve a list of counties
region_df = get_regions('county')
region_df

,region_id,region_type,region
None,,,
0,999,county,Durham County; NC; Durham-Chapel Hill
1,998,county,Duplin County; NC
2,997,county,Dubois County; IN; Jasper
3,995,county,Donley County; TX
4,993,county,Dimmit County; TX
...,...,...,...
2886,1003,county,Elmore County; AL; Montgomery
2887,1002,county,Elbert County; GA
2888,1001,county,Elbert County; CO; Denver-Aurora-Lakewood


In [108]:
# Massaging zillow county names

region_df['region']=region_df['region'].str.split(';', n=-1, expand=True)[0]

In [109]:
print(region_df)

     region_id region_type         region
None                                     
0          999      county  Durham County
1          998      county  Duplin County
2          997      county  Dubois County
3          995      county  Donley County
4          993      county  Dimmit County
...        ...         ...            ...
2886      1003      county  Elmore County
2887      1002      county  Elbert County
2888      1001      county  Elbert County
2889      1000      county  Echols County
2890       100      county    Bibb County

[2891 rows x 3 columns]


In [110]:
#list(region_df['region_id'])

In [9]:
# The actual API call using the SDK.
# Instructions can be found here https://data.nasdaq.com/databases/ZILLOW/usage/quickstart/python
# Replace 'quandl' w/ 'nasdaqdatalink

# '''data = nasdaqdatalink.get_table('ZILLOW/DATA', paginate=True, indicator_id='ZSFH', region_id=list(region_df['region_id']))'''

data = nasdaqdatalink.export_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=list(region_df['region_id']),filename='db.zip')

DataLinkError: (Status 413) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@data.nasdaq.com.

In [17]:
# Unzipping database from API call

shutil.unpack_archive('db.zip')

In [19]:
# Reading in Database

zillow_data=pd.read_csv(
    Path('ZILLOW_DATA_d5d2ff90eb7172dbde848ea36de12dfe.csv')
)

In [24]:
# Displaying database

display(zillow_data.head(10))
display(zillow_data.tail(10))

,indicator_id,region_id,date,value
0,ZSFH,100,2007-11-30,123760.0
1,ZSFH,100,2007-12-31,123754.0
2,ZSFH,100,2008-01-31,123605.0
3,ZSFH,100,2008-02-29,123393.0
4,ZSFH,100,2008-03-31,123095.0
5,ZSFH,100,2008-04-30,123018.0
6,ZSFH,100,2008-05-31,122855.0
7,ZSFH,100,2008-06-30,122883.0
8,ZSFH,100,2008-07-31,122853.0
9,ZSFH,100,2008-08-31,123015.0


,indicator_id,region_id,date,value
669306,ZSFH,999,2021-09-30,346041.0
669307,ZSFH,999,2021-10-31,355160.0
669308,ZSFH,999,2021-11-30,361432.0
669309,ZSFH,999,2021-12-31,368325.0
669310,ZSFH,999,2022-01-31,380061.0
669311,ZSFH,999,2022-02-28,390111.0
669312,ZSFH,999,2022-03-31,401621.0
669313,ZSFH,999,2022-04-30,411421.0
669314,ZSFH,999,2022-05-31,422028.0
669315,ZSFH,999,2022-06-30,430509.0


In [27]:
# Read in county data with coordinates

county_coordinates_df=pd.read_csv(
    Path('counties_w_coordinates.csv')
)

county_coordinates_df

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,+32.536382°,–86.644490°
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",+30.659218°,–87.746067°
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,+31.870670°,–85.405456°
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,3139.0,WY,56037.0,Sweetwater,Green River,"43,806","27,004.90","10,426.65",166.887,64.436,"27,171.78","10,491.09",+41.660339°,–108.875676°
3140,3140.0,WY,56039.0,Teton,Jackson,"21,294","10,347.98","3,995.38",572.266,220.953,"10,920.25","4,216.33",+44.049321°,–110.588102°
3141,3141.0,WY,56041.0,Uinta,Evanston,"21,118","5,390.45","2,081.26",16.342,6.31,"5,406.79","2,087.57",+41.284726°,–110.558947°
3142,3142.0,WY,56043.0,Washakie,Worland,"8,533","5,797.82","2,238.55",10.762,4.155,"5,808.58","2,242.70",+43.878831°,–107.669052°
